## Diet problem

A canteen has to plan the composition of the meals that it provides. A meal can be composed of the types of food indicated in the following table. Costs, in Euro per hg, and availabilities, in hg, are also indicated.

Food|Cost|Availability
----|----|------------
Bread|0.1|4
Milk|0.5|3
Eggs|0.12|1
Meat|0.9|2
Cake|1.3|2

A meal must contain at least the following amount of each nutrient

Nutrient|Minimal quantity
--------|----------------
Calories|600 cal
Proteins|50 g
Calcium|0.7 g

Each hg of each type of food contains to following amount of nutrients

Food|Calories|Proteins|Calcium
----|--------|--------|-------
Bread|30 cal|5 g|0.02 g
Milk|50 cal|15 g|0.15 g
Eggs|150 cal|30 g|0.05 g
Meat|180 cal|90 g|0.08 g
Cake|400 cal|70 g|0.01 g

Give a linear programming formulation for the problem of finding a diet (a meal) of minimum total cost which satisfies the minimum nutrient requirements.

In [13]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

5236.07s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# We need to import the package mip

import mip

In [25]:
# Food
I = {'Bread', 'Milk', 'Eggs', 'Meat', 'Cake'}

# Nutrients
J = {'Calories', 'Proteins', 'Calcium'}

# Cost in Euro per hg of food
c = {'Bread':0.1, 'Milk':0.5, 'Eggs':0.12, 'Meat':0.9, 'Cake':1.3}

# Availability per hg of food
q = {'Bread':4, 'Milk':3, 'Eggs':1, 'Meat':2, 'Cake':2}

# Minimum nutrients 
b = {'Calories':600, 'Proteins':50, 'Calcium':0.7}

# Nutrients per hf of food
a = {('Bread', 'Calories'):30, ('Milk', 'Calories'):50, ('Eggs', 'Calories'):150, ('Meat', 'Calories'):180, ('Cake', 'Calories'):400,
    ('Bread', 'Proteins'):5, ('Milk', 'Proteins'):15, ('Eggs', 'Proteins'):30, ('Meat', 'Proteins'):90, ('Cake', 'Proteins'):70,
    ('Bread', 'Calcium'):0.02, ('Milk', 'Calcium'):0.15, ('Eggs', 'Calcium'):0.05, ('Meat', 'Calcium'):0.08, ('Cake', 'Calcium'):0.01}

Now we create an empty model and add the variables

In [26]:
# Define a model
model = mip.Model()

# Define variables optional parameters
x = {i : model.add_var(name = i) for i in I}

In [5]:
print(x['Bread'])

Bread


In [6]:
print(x)

{'Eggs': <mip.entities.Var object at 0x7aa7cf47daf0>, 'Bread': <mip.entities.Var object at 0x7aa7cf47da90>, 'Milk': <mip.entities.Var object at 0x7aa7ce1037c0>, 'Cake': <mip.entities.Var object at 0x7aa7ce103850>}


Let's write the objective function: in the general case, it can be written as a sum over the set of models.

In [27]:
# Define the objective function
# in our ci and xi
model.objective = mip.minimize(mip.xsum(c[i]*x[i] for i in I))

# alternative 

# model.objective = mip.minimize(c[0]*x[0] + c[1]*x[1] + c[2]*x[2] + c[3]*x[3] + c[4]*x[4]) 

The constraints can be generated in loops:

In [28]:
# CONSTRAINTS

# Availability constraint
for i in I:
  model.add_constr(x[i] <= q[i])

# Minum nutrients
for j in J:
  model.add_constr(mip.xsum(a[i,j]*x[i] for i in I )>=b[j])

The model is complete. Let's solve it and print the optimal solution.

In [29]:
# Optimizing command
model.optimize()

Coin0507I Presolve determined that the problem was infeasible with tolerance of 1e-08
Clp3003W Analysis indicates model infeasible or unbounded
Clp0001I Primal infeasible - objective value 17.62
Clp0032I PrimalInfeasible objective 17.62 - 4 iterations time 0.012
Starting solution of the Linear programming problem using Dual Simplex



<OptimizationStatus.OPTIMAL: 0>

In [30]:
# Optimal objective function value
model.objective.x

3.37

In [31]:
# Printing the variables values
for i in model.vars:
  print(i.name)
  print(i.x)

Eggs
1.0
Bread
3.9999999999999996
Cake
0.0
Meat
1.5000000000000002
Milk
3.0


## Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10


In [32]:
# Set of oil types
I = range(4)

# Set of gasoline types
J = {'A', 'B', 'C'}

# Unit cost for oil of type i 
c = {0:0.9,1:7,2:12,3:6}

# Availability of oil type i
b ={0:5000,1:2400,2:4000,3:1500}

# Price of gasoline of type j
r = {'A':12, 'B':18, 'C':10}

# Maximum quantity (percentage) of oil
q_max = {}
for i in I:
  for j in J:
    q_max[(str(i),j)] = 1
q_max[('2','A')] = 0.3
q_max[('1','C')] = 0.5

# Minimum quantity (percentage) of oil
q_min = {}
for i in I:
  for j in J:
    q_min[(str(i),j)] = 0
q_min[('1','A')] = 0.2
q_min[('2','B')] = 0.4

Now we create an empty model and add the variables

In [33]:
# Define a model
model2 = mip.Model()

# Define variables
x = {(str(i),j):model2.add_var(name=str(i)+','+j)  for i in I for j in J}
y = {j:model2.add_var(name=j, lb=0) for j in J}

In [34]:
# Define the objective function
model2.objective = mip.maximize(mip.xsum(r[j]*y[j] for j in J) - mip.xsum(c[i]*x[(str(i),j)] for i in I for j in J))

In [35]:
# CONSTRAINTS
# Availability constraint
for i in I:
  model2.add_constr(mip.xsum(x[str(i),j] for j in J) <= b[i])

# Conservation constraint
for j in J:
  model2.add_constr(mip.xsum(x[str(i),j] for i in I) == y[j])

# Maximum quantity
for i in I:
  for j in J:
      model2.add_constr(x[str(i),j] <= q_max[(str(i),j)]*y[j])
    
# Minimum quantity
for i in I:
  for j in J:
    model2.add_constr(x[str(i),j] >= q_min[(str(i),j)]*y[j]) 

The model is complete. Let's solve it and print the optimal solution.

In [36]:
# Optimizing command
model2.optimize()

Coin0506I Presolve 2 (-6) rows, 5 (0) columns and 10 (-10) elements
Clp0006I 0  Obj 2.3749999 Primal inf 2.0358316 (2)
Clp0000I Optimal - objective value 3.37
Coin0511I After Postsolve, objective 3.37, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 3.37 - 1 iterations time 0.002, Presolve 0.00
Starting solution of the Linear programming problem using Dual Simplex



<OptimizationStatus.OPTIMAL: 0>

In [37]:
# Optimal objective function value
model2.objective.x

136500.0

In [38]:
# Printing the variables values
for i in model2.vars:
  print(i.name)
  print(i.x)
  print('-----')

0,B
4500.0
-----
0,A
500.0
-----
0,C
0.0
-----
1,B
0.0
-----
1,A
2400.0
-----
1,C
0.0
-----
2,B
4000.0
-----
2,A
0.0
-----
2,C
0.0
-----
3,B
1500.0
-----
3,A
0.0
-----
3,C
0.0
-----
B
10000.0
-----
A
2900.0
-----
C
0.0
-----
